In [ ]:
import os

from google.colab import drive
gdrive_path = '/content/drive/'
drive.mount(gdrive_path)
colab_path = 'drive/MyDrive/Colab Notebooks'
# os.chdir(colab_path)

data_path = os.path.join(colab_path, 'datasets/')
os.makedirs(data_path, exist_ok=True)

weights_path = os.path.join(colab_path, 'weights/')
os.makedirs(weights_path, exist_ok=True)

Mounted at /content/drive/


In [ ]:
#모듈불러오기
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
from torch import optim
from torch.optim.lr_scheduler import StepLR

# dataset and transformation
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import os

# display images
from torchvision import utils
import matplotlib.pyplot as plt
%matplotlib inline

# utils
import numpy as np
from torchsummary import summary
import time
import copy

import torchvision.transforms as transforms

In [ ]:
#장비
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
#HyperParameter
batch_size = 32
epoch_num = 15
learning_rate = 0.0001

In [ ]:
transform = transforms.Compose([transforms.Resize(224), transforms.ToTensor()])

In [ ]:
#데이터 불러오기
train_ds = datasets.STL10(root=data_path, split='train', download=True, transform=transform)
val_ds = datasets.STL10(root=data_path, split='test', download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
print('number of training data: ', len(train_ds))
print('number of test data: ', len(val_ds))

number of training data:  5000
number of test data:  8000


In [ ]:
#미니 배치 구성하기
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=batch_size, shuffle=True)

In [ ]:
train_dl.dataset

Dataset STL10
    Number of datapoints: 5000
    Root location: drive/MyDrive/Colab Notebooks/datasets/
    Split: train
    StandardTransform
Transform: Compose(
               Resize(size=224, interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
           )

In [ ]:
#VGG19모델 구성하기
class VGG(nn.Module):
  def __init__(self):
    super(VGG, self).__init__()
    kernel_size=(3,3)
    stride=(1,1)
    padding=(1,1)
    self.conv1 = nn.Conv2d(3, 64, kernel_size=kernel_size, stride=stride, padding=padding) #(in_channels, out_channnels, kernel_size, stride, padding)
    self.conv2 = nn.Conv2d(64, 64, kernel_size=kernel_size, stride=stride, padding=padding)

    self.conv3 = nn.Conv2d(64, 128, kernel_size=kernel_size, stride=stride, padding=padding)
    self.conv4 = nn.Conv2d(128, 128, kernel_size=kernel_size, stride=stride, padding=padding)

    self.conv5 = nn.Conv2d(128, 256, kernel_size=kernel_size, stride=stride, padding=padding)
    self.conv6 = nn.Conv2d(256, 256, kernel_size=kernel_size, stride=stride, padding=padding)
    self.conv7 = nn.Conv2d(256, 256, kernel_size=kernel_size, stride=stride, padding=padding)
    self.conv8 = nn.Conv2d(256, 256, kernel_size=kernel_size, stride=stride, padding=padding)

    self.conv9 = nn.Conv2d(256, 512, kernel_size=kernel_size, stride=stride, padding=padding)
    self.conv10 = nn.Conv2d(512, 512, kernel_size=kernel_size, stride=stride, padding=padding)
    self.conv11 = nn.Conv2d(512, 512, kernel_size=kernel_size, stride=stride, padding=padding)
    self.conv12 = nn.Conv2d(512, 512, kernel_size=kernel_size, stride=stride, padding=padding)

    self.conv13 = nn.Conv2d(512, 512, kernel_size=kernel_size, stride=stride, padding=padding)
    self.conv14 = nn.Conv2d(512, 512, kernel_size=kernel_size, stride=stride, padding=padding)
    self.conv15 = nn.Conv2d(512, 512, kernel_size=kernel_size, stride=stride, padding=padding)
    self.conv16 = nn.Conv2d(512, 512, kernel_size=kernel_size, stride=stride, padding=padding)
    
    self.fc1 = nn.Linear(25088, 4096)
    self.fc2 = nn.Linear(4096, 4096)
    self.fc3 = nn.Linear(4096, 1000)
    self.fc4 = nn.Linear(1000, 10)

  def forward(self, x)  :
    x = self.conv1(x)
    x = F.relu(x)
    x = self.conv2(x)
    x = F.relu(x)

    x = F.max_pool2d(x,kernel_size=2, stride=2, padding=0, dilation=1)

    x = self.conv3(x)
    x = F.relu(x)
    x = self.conv4(x)
    x = F.relu(x)

    x = F.max_pool2d(x,kernel_size=2, stride=2, padding=0, dilation=1)

    x = self.conv5(x)
    x = F.relu(x)
    x = self.conv6(x)
    x = F.relu(x)
    x = self.conv7(x)
    x = F.relu(x)
    x = self.conv8(x)
    x = F.relu(x)

    x = F.max_pool2d(x,kernel_size=2, stride=2, padding=0, dilation=1)

    x = self.conv9(x)
    x = F.relu(x)
    x = self.conv10(x)
    x = F.relu(x)
    x = self.conv11(x)
    x = F.relu(x)
    x = self.conv12(x)
    x = F.relu(x)

    x = F.max_pool2d(x,kernel_size=2, stride=2, padding=0, dilation=1)

    x = self.conv13(x)
    x = F.relu(x)
    x = self.conv14(x)
    x = F.relu(x)
    x = self.conv15(x)
    x = F.relu(x)
    x = F.max_pool2d(x,kernel_size=2, stride=2, padding=0, dilation=1)
    x = torch.flatten(x, 1)

    x = self.fc1(x)
    x = F.relu(x)
    x = self.fc2(x)
    x = F.relu(x)
    x = self.fc3(x)
    x = F.relu(x)
    x = self.fc4(x)
    x = F.relu(x)

    output = F.log_softmax(x, dim=1)
    return output

In [ ]:
#Optimizer 및 손실 함수 정의
model= VGG().to(device)
optimizer= optim.Adam(model.parameters(), lr= learning_rate)
criterion= nn.CrossEntropyLoss()#분류

In [ ]:
print(model)

VGG(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv6): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv7): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv9): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv10): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv11): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv12): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv13): Conv2d(512, 512, kernel_size=(3, 3), s

In [ ]:
#모델학습
model.train()
i=0
for epoch in range(epoch_num):
  for data, target in train_dl:
    data = data.to(device)
    target = target.to(device)
    
    optimizer.zero_grad()#가중치 초기화
    
    output = model(data)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()
    
    if i %1000==0:
      print('Train step: {}\tloss: {:,.3f}'.format(i, loss.item()))

Train step: 0	loss: 2.303
Train step: 0	loss: 2.302
Train step: 0	loss: 2.307
Train step: 0	loss: 2.303
Train step: 0	loss: 2.298
Train step: 0	loss: 2.304
Train step: 0	loss: 2.308
Train step: 0	loss: 2.310
Train step: 0	loss: 2.303
Train step: 0	loss: 2.305
Train step: 0	loss: 2.303
Train step: 0	loss: 2.298
Train step: 0	loss: 2.305
Train step: 0	loss: 2.300
Train step: 0	loss: 2.298
Train step: 0	loss: 2.302
Train step: 0	loss: 2.302
Train step: 0	loss: 2.302
Train step: 0	loss: 2.299
Train step: 0	loss: 2.305
Train step: 0	loss: 2.308
Train step: 0	loss: 2.303
Train step: 0	loss: 2.303
Train step: 0	loss: 2.302
Train step: 0	loss: 2.299
Train step: 0	loss: 2.301
Train step: 0	loss: 2.306
Train step: 0	loss: 2.307
Train step: 0	loss: 2.306
Train step: 0	loss: 2.303
Train step: 0	loss: 2.303
Train step: 0	loss: 2.304
Train step: 0	loss: 2.303
Train step: 0	loss: 2.302
Train step: 0	loss: 2.307
Train step: 0	loss: 2.307
Train step: 0	loss: 2.304
Train step: 0	loss: 2.303
Train step: 

In [ ]:
#모델학습
model.train()
i=0
for epoch in range(epoch_num):
  for data, target in train_dl:
    data = data.to(device)
    target = target.to(device)
    
    optimizer.zero_grad()#가중치 초기화
    
    output = model(data)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()
    
    if i %1000==0:
      print('Train step: {}\tloss: {:,.3f}'.format(i, loss.item()))
    i+=1
    break
  break

Train step: 0	loss: 2.303
